# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

In [1]:
from langchain.agents import create_agent

In [2]:
agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[],
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [3]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Why do programmers prefer dark mode?

Because light attracts bugs.


## values
You have seen this streaming mode in our examples so far. 

In [4]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why did the scarecrow win an award? Because he was outstanding in his field.

Want another?


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [5]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

Sunrise tiptoes across the sill,
Warm and soft, the world is still.
Pancake giggles, syrup smiles,
Little footprints in sticky piles.

We build a fort of blankets and dreams,
Whispering secrets in sunlit beams.
Outside the garden hums and plays,
Bees like tiny golden rays.

Hand in hand down the winding lane,
Evening brings a gentle tune,
Moonlight tucked beneath the spoon.

Home is where our bright hearts meet—
Full of love and small, sweet feet.

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [6]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='2ae668e9-aa7f-4643-a94a-ea320c737fd1')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='2ae668e9-aa7f-4643-a94a-ea320c737fd1'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 132, 'total_tokens': 284, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-COQf7uMzBXq1SekpyTSBWCCCdMA4l', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--8d054820-aaa9-4734-9306-ebf14eecf3df-0', tool_calls=[{'name': 'get_

In [7]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco
Acquired data for city: San Francisco


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [8]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco, CA
Acquired data for city: San Francisco, CA
